In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [2]:
device = 'cpu'

num_epochs = 50
batch_size = 4
learning_rate = 0.01

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = train_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader =  torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')



Files already downloaded and verified
Files already downloaded and verified


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        #forward
        outputs = model(images)
        #loss
        loss = criterion(outputs, labels)
        
        #back
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2500 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
            

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        #value and index which is why we dont need first
        _, predictions = torch.max(output, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
    

epoch 1 / 50, step 2500/2500, loss = 1.9192
epoch 2 / 50, step 2500/2500, loss = 1.7095
epoch 3 / 50, step 2500/2500, loss = 1.6443
epoch 4 / 50, step 2500/2500, loss = 2.1179
epoch 5 / 50, step 2500/2500, loss = 2.0658
epoch 6 / 50, step 2500/2500, loss = 1.3464
epoch 7 / 50, step 2500/2500, loss = 1.3869
epoch 8 / 50, step 2500/2500, loss = 0.8525
epoch 9 / 50, step 2500/2500, loss = 1.0642
epoch 10 / 50, step 2500/2500, loss = 0.9379
epoch 11 / 50, step 2500/2500, loss = 1.7319
epoch 12 / 50, step 2500/2500, loss = 0.4514
epoch 13 / 50, step 2500/2500, loss = 0.5409
epoch 14 / 50, step 2500/2500, loss = 1.3867
epoch 15 / 50, step 2500/2500, loss = 1.5096
epoch 16 / 50, step 2500/2500, loss = 0.8909
epoch 17 / 50, step 2500/2500, loss = 0.5903
epoch 18 / 50, step 2500/2500, loss = 1.5550
epoch 19 / 50, step 2500/2500, loss = 0.9789
epoch 20 / 50, step 2500/2500, loss = 0.3716
epoch 21 / 50, step 2500/2500, loss = 0.1900
epoch 22 / 50, step 2500/2500, loss = 1.3453
epoch 23 / 50, step

In [5]:
acc = n_correct/n_samples * 100.0
print(acc, '%')

94.31 %
